In [1]:
import pandas as pd
import numpy as np

In [2]:
# read master roster raw file
fileFolder = r'..\05 OMD\Master Roster'
fileName = 'latest_roster_raw.xlsx'
file = '\\'.join([fileFolder, fileName])
df = pd.read_excel(file, sheet_name = 'Roster')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Week Start                      1565 non-null   datetime64[ns]
 1   Site                            1565 non-null   object        
 2   Account                         1565 non-null   object        
 3   Line of Business                1565 non-null   object        
 4   Employee ID                     1564 non-null   float64       
 5   eWFM ID                         1564 non-null   object        
 6   Kronos ID                       1564 non-null   object        
 7   Global ID                       1560 non-null   float64       
 8   VZID                            1559 non-null   object        
 9   Last Name                       1565 non-null   object        
 10  First Name                      1565 non-null   object        
 11  Comp

In [3]:
# select columns
cols = ['CIMID', 'Week Start', 'Site', 'Employee ID', 'Kronos ID', 'Global ID', 'VZID', 'Last Name',
        'First Name', 'Complete Name', 'Supervisor', 'Manager', 'Trainer', 'Wave', 'Position', 'Role', 
        'Department', 'Productive/Unproductive', 'Work Status', 'Work Status II', 'SYKES Hire Date',
        'Account Start Date','Nesting Date', 'Live Date', 'Term Date', 'Tenure Category', 
        'Tenure with Sykes', 'Situational Presence']

df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CIMID                    1565 non-null   object        
 1   Week Start               1565 non-null   datetime64[ns]
 2   Site                     1565 non-null   object        
 3   Employee ID              1564 non-null   float64       
 4   Kronos ID                1564 non-null   object        
 5   Global ID                1560 non-null   float64       
 6   VZID                     1559 non-null   object        
 7   Last Name                1565 non-null   object        
 8   First Name               1565 non-null   object        
 9   Complete Name            1565 non-null   object        
 10  Supervisor               1565 non-null   object        
 11  Manager                  1565 non-null   object        
 12  Trainer                  1467 non-

In [4]:
# Fill NAT of Date cols
df['Live Date'].fillna(df['Account Start Date'] + pd.Timedelta(60), inplace = True )

dt_cols = ['SYKES Hire Date', 'Account Start Date', 'Nesting Date', 'Live Date', 'Term Date']
for col in dt_cols :
    df[col].fillna('1900/1/1', inplace = True )
    print(col + ': ', df[col].isna().sum())

SYKES Hire Date:  0
Account Start Date:  0
Nesting Date:  0
Live Date:  0
Term Date:  0


In [5]:
df.loc[df['Employee ID'] == 10609395, 'Live Date'] = '1900-1-1'
ren = df.loc[df['Employee ID'] == 10609395]

In [6]:
# clean data types
df['CIMID'] = pd.to_numeric(df['CIMID'], errors = 'coerce').fillna(0).astype('float64')
df['Employee ID'] = pd.to_numeric(df['Employee ID'], errors = 'coerce').fillna(0).astype('float64')
df['Global ID'] = pd.to_numeric(df['Global ID'], errors = 'coerce').fillna(0)
df['Wave'] = pd.to_numeric(df['Wave'], errors = 'coerce')


for col in dt_cols :
    df[col] = pd.to_datetime(df[col]).dt.normalize()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1565 entries, 0 to 1564
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CIMID                    1565 non-null   float64       
 1   Week Start               1565 non-null   datetime64[ns]
 2   Site                     1565 non-null   object        
 3   Employee ID              1565 non-null   float64       
 4   Kronos ID                1564 non-null   object        
 5   Global ID                1565 non-null   float64       
 6   VZID                     1559 non-null   object        
 7   Last Name                1565 non-null   object        
 8   First Name               1565 non-null   object        
 9   Complete Name            1565 non-null   object        
 10  Supervisor               1565 non-null   object        
 11  Manager                  1565 non-null   object        
 12  Trainer                  1467 non-

In [7]:
df[df['Live Date'].isna()]

,CIMID,Week Start,Site,Employee ID,Kronos ID,Global ID,VZID,Last Name,First Name,Complete Name,...,Work Status,Work Status II,SYKES Hire Date,Account Start Date,Nesting Date,Live Date,Term Date,Tenure Category,Tenure with Sykes,Situational Presence


In [8]:
# check dups
df = df.drop_duplicates(['Global ID'], keep ='last')
df.duplicated(['Global ID']).sum()

0

In [9]:
# clean Wave column
pd.set_option("display.max_rows", None, "display.max_columns", None)

df['Wave'] = df['Wave'].apply(np.floor)
df['Wave'] = df['Wave'].fillna(-99)
df['Wave'] = df['Wave'].astype('float64')
df['Wave'].value_counts(ascending = True)

-99.0     2
 34.0     6
 4.0      7
 1.0      9
 8.0      9
 19.0    10
 23.0    11
 12.0    12
 2.0     12
 9.0     12
 7.0     13
 18.0    14
 24.0    14
 6.0     14
 5.0     14
 3.0     14
 15.0    15
 11.0    15
 17.0    16
 10.0    17
 16.0    17
 22.0    17
 20.0    18
 25.0    18
 13.0    18
 26.0    19
 40.0    19
 30.0    19
 32.0    20
 27.0    20
 33.0    21
 14.0    21
 41.0    23
 60.0    23
 35.0    23
 43.0    23
 21.0    23
 51.0    24
 45.0    24
 28.0    24
 29.0    24
 37.0    24
 36.0    24
 52.0    24
 61.0    25
 42.0    25
 44.0    25
 46.0    25
 48.0    25
 49.0    25
 63.0    25
 31.0    25
 39.0    26
 53.0    26
 0.0     27
 47.0    27
 38.0    27
 50.0    33
 59.0    35
 58.0    40
 62.0    45
 64.0    51
 55.0    64
 57.0    64
 54.0    70
 56.0    79
Name: Wave, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1560 entries, 0 to 1564
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CIMID                    1560 non-null   float64       
 1   Week Start               1560 non-null   datetime64[ns]
 2   Site                     1560 non-null   object        
 3   Employee ID              1560 non-null   float64       
 4   Kronos ID                1560 non-null   object        
 5   Global ID                1560 non-null   float64       
 6   VZID                     1558 non-null   object        
 7   Last Name                1560 non-null   object        
 8   First Name               1560 non-null   object        
 9   Complete Name            1560 non-null   object        
 10  Supervisor               1560 non-null   object        
 11  Manager                  1560 non-null   object        
 12  Trainer                  1462 non-

In [11]:
df.to_excel(r'..\Master Roster\clean_roster_frm_python.xlsx',
           index = False)

In [12]:
# MS Access SQL query
# read Roster table from MS Access BGCO DB
import pyodbc

db = r'..\Documents\BGCO DB.accdb;'
driver = r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
conn = pyodbc.connect(driver + 'DBQ={}'.format(db) )
cursor = conn.cursor()

select_all_query = '''
    SELECT * 
    FROM Roster
'''

df_db = pd.read_sql(select_all_query, conn)
cursor.commit()                     

In [13]:
# del Roster data
del_all_rows_query = '''
    DELETE 
    FROM Roster
'''

cursor.execute(del_all_rows_query)
df_db = pd.read_sql(select_all_query, conn)
cursor.commit()                     

In [14]:
df.columns

Index(['CIMID', 'Week Start', 'Site', 'Employee ID', 'Kronos ID', 'Global ID',
       'VZID', 'Last Name', 'First Name', 'Complete Name', 'Supervisor',
       'Manager', 'Trainer', 'Wave', 'Position', 'Role', 'Department',
       'Productive/Unproductive', 'Work Status', 'Work Status II',
       'SYKES Hire Date', 'Account Start Date', 'Nesting Date', 'Live Date',
       'Term Date', 'Tenure Category', 'Tenure with Sykes',
       'Situational Presence'],
      dtype='object')

In [15]:
# insert clean df to Roster table
insert_df = '''
    INSERT INTO 
        Roster(
            CIMID, [Week Start], [Site], [Employee ID], [Kronos ID], [Global ID], [VZID], [Last Name], 
            [First Name], [Complete Name], [Supervisor], [Manager], [Trainer], [Wave], [Position], [Role], 
            [Department], [Productive/Unproductive], [Work Status], [Work Status II], [SYKES Hire Date], 
            [Account Start Date], [Nesting Date], [Live Date], [Term Date], [Tenure Category], 
            [Tenure with Sykes], [Situational Presence]
        ) 
        VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''
for idx, row in df.iterrows():
    cursor.execute(insert_df, row['CIMID'], row['Week Start'], row['Site'], row['Employee ID'], row['Kronos ID'], 
                   row['Global ID'], row['VZID'], row['Last Name'], row['First Name'], row['Complete Name'], 
                   row['Supervisor'], row['Manager'], row['Trainer'], row['Wave'], row['Position'], row['Role'], 
                   row['Department'], row['Productive/Unproductive'], row['Work Status'], row['Work Status II'], 
                   row['SYKES Hire Date'], row['Account Start Date'], row['Nesting Date'], row['Live Date'], 
                   row['Term Date'], row['Tenure Category'],row['Tenure with Sykes'],row['Situational Presence']
                  ) 

cursor.commit()

In [16]:
# read updated Roster table
df_db = pd.read_sql(select_all_query, conn)
conn.close()

df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   CIMID                    1560 non-null   float64       
 1   Week Start               1560 non-null   datetime64[ns]
 2   Site                     1560 non-null   object        
 3   Account                  0 non-null      object        
 4   Line of Business         0 non-null      object        
 5   Employee ID              1560 non-null   float64       
 6   eWFM ID                  0 non-null      object        
 7   Kronos ID                1560 non-null   object        
 8   Global ID                1560 non-null   float64       
 9   VZID                     1560 non-null   object        
 10  Last Name                1560 non-null   object        
 11  First Name               1560 non-null   object        
 12  Complete Name            1560 non-

In [17]:
# MS Access SQL query for main query

db = r'..\Documents\BGCO DB.accdb;'
driver = r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};'
conn = pyodbc.connect(driver + 'DBQ={}'.format(db) )
main_query = ''' 
    SELECT *
    FROM 0_bgco_main_query
    WHERE month_year = '2022-02' or month_year = '2022-03'
    ORDER BY stat_date ;
'''
                     
df_main_query = pd.read_sql(main_query, conn)

conn.close()                     

df_main_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14374 entries, 0 to 14373
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   CIM_Worker_ID                 14374 non-null  float64       
 1   rpt_mth                       14374 non-null  datetime64[ns]
 2   month_year                    14374 non-null  object        
 3   stat_date                     14374 non-null  datetime64[ns]
 4   EMPLOYEE_NAME                 14374 non-null  object        
 5   RepHandledCalls               14374 non-null  float64       
 6   TransferCalls                 14374 non-null  float64       
 7   nps_promoter                  14374 non-null  float64       
 8   nps_detractor                 14374 non-null  float64       
 9   nps_surveys                   14374 non-null  float64       
 10  TechTransfers                 14374 non-null  float64       
 11  Net_OCC                     